In [20]:
import cv2
import numpy as np
from PIL import Image
import math

In [21]:
def gaussian1D(sigma):
    a = sigma
    
    if a % 2 == 0: #mask size가 짝수일 경우 1을 키운다
        a+=1
    
    a2 = a/2
    filter_size = np.array(range(-int(a/2),int(a/2)+1))
    result = np.zeros(len(filter_size))
    
    for i in range(len(filter_size)):
        x = i-a2
        result[i] =  float(np.exp(-(x**2)/(2*sigma**2)) / (2*3.14*sigma**2)) #가우시안 공식
    
    total = np.sum(result)
    result/=total
    
    return result

In [22]:
def gaussian2D(sigma):
    # 두 벡터를 외적하여 x,y 2차원 mask를 생성
    result = np.outer(gaussian1D(sigma),gaussian1D(sigma))
    total = np.sum(result)
    result/=total
    
    return result

In [24]:
def convolution_color(array, f):
    a = (len(f)-1)//2
    # boundery를 0으로 채운다
    img_pad = np.pad(array, ((a,a),(a,a),(0,0)),'constant', constant_values=0)
    
    #convolution을 위해 2번 뒤집는다
    mask = np.rot90(f)
    mask = np.rot90(mask)
    
    #color img 이므로 3차원으로 배열 설정
    result_img = np.zeros((len(array),len(array[0]),3))
    result_img = result_img.astype('float32')
    
    for i in range(len(result_img)):
        for j in range(len(result_img[0])):
            result_img[i][j][0] = np.sum(img_pad[i:i+len(f), j:j+len(f),0]*mask)
            result_img[i][j][1] = np.sum(img_pad[i:i+len(f), j:j+len(f),1]*mask)
            result_img[i][j][2] = np.sum(img_pad[i:i+len(f), j:j+len(f),2]*mask)
    return result_img

In [25]:
def real_convolution(array, sigma):
    return convolution(array, gaussian2D(sigma))

In [26]:
def real_convolution_color(array, sigma):
    return convolution_color(array, gaussian2D(sigma))

In [27]:
print(gaussian2D(7))

[[0.01730548 0.01839811 0.01916458 0.01955972 0.01955972 0.01916458
  0.01839811]
 [0.01839811 0.01955972 0.02037459 0.02079467 0.02079467 0.02037459
  0.01955972]
 [0.01916458 0.02037459 0.02122341 0.02166099 0.02166099 0.02122341
  0.02037459]
 [0.01955972 0.02079467 0.02166099 0.02210759 0.02210759 0.02166099
  0.02079467]
 [0.01955972 0.02079467 0.02166099 0.02210759 0.02210759 0.02166099
  0.02079467]
 [0.01916458 0.02037459 0.02122341 0.02166099 0.02166099 0.02122341
  0.02037459]
 [0.01839811 0.01955972 0.02037459 0.02079467 0.02079467 0.02037459
  0.01955972]]


In [28]:
img = Image.open('Salt&pepper noise.png')

img_arr = np.asarray(img)
img_arr = img_arr.astype('float32')

img_result = real_convolution_color(img_arr, 10)
img_result = img_result.astype('uint8')
img_result = Image.fromarray(img_result)

img_result.show()